In [1]:
import json


class Direccion:
    def __init__(self, calle: str, ciudad: str, codigo_postal: str):
        self.calle = calle
        self.ciudad = ciudad
        self.codigo_postal = codigo_postal

    def __str__(self):
        return f"{self.calle}, {self.ciudad}, {self.codigo_postal}"

    def to_dict(self):
        return {
            "calle": self.calle,
            "ciudad": self.ciudad,
            "codigo_postal": self.codigo_postal,
        }

    @classmethod
    def from_dict(cls, data):
        return cls(data["calle"], data["ciudad"], data["codigo_postal"])


class Cliente:
    def __init__(self, nombre: str, direccion: Direccion):
        self._nombre = None
        self.nombre = nombre
        self.direccion = direccion

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if not isinstance(value, str) or not value.strip():
            raise ValueError("El nombre del cliente debe ser una cadena no vacía.")
        self._nombre = value.strip()

    def __str__(self):
        return f"Cliente: {self.nombre}, Dirección: {self.direccion}"

    def to_dict(self):
        return {
            "nombre": self.nombre,
            "direccion": self.direccion.to_dict(),
        }

    @classmethod
    def from_dict(cls, data):
        direccion = Direccion.from_dict(data["direccion"])
        return cls(data["nombre"], direccion)


class Paquete:
    def __init__(self, peso: float, precio_base: float):
        self._peso = None
        self._precio_base = None
        self.peso = peso
        self.precio_base = precio_base
        self._estado = "pendiente"

    @property
    def peso(self):
        return self._peso

    @peso.setter
    def peso(self, value):
        if value <= 0:
            raise ValueError("El peso debe ser un número positivo.")
        self._peso = value

    @property
    def precio_base(self):
        return self._precio_base

    @precio_base.setter
    def precio_base(self, value):
        if value < 0:
            raise ValueError("El precio base no puede ser negativo.")
        self._precio_base = value

    def set_estado(self, nuevo_estado: str):
        estados_validos = ["pendiente", "en tránsito", "entregado"]
        if nuevo_estado not in estados_validos:
            raise ValueError(f"Estado inválido. Debe ser uno de {estados_validos}")
        self._estado = nuevo_estado

    def get_estado(self):
        return self._estado

    def calcular_precio(self):
        return self.precio_base

    def __str__(self):
        return f"Paquete {self.peso}kg - Precio Base: ${self.precio_base:.2f} - Estado: {self._estado}"

    def to_dict(self):
        return {
            "tipo": self.__class__.__name__,
            "peso": self.peso,
            "precio_base": self.precio_base,
            "estado": self._estado,
        }

    @classmethod
    def from_dict(cls, data):
        tipo = data.get("tipo", "Paquete")
        if tipo == "PaqueteExpress":
            paquete = PaqueteExpress(data["peso"], data["precio_base"], data.get("recargo", 0.5))
        elif tipo == "PaqueteEstandar":
            paquete = PaqueteEstandar(data["peso"], data["precio_base"])
        else:
            paquete = Paquete(data["peso"], data["precio_base"])
        paquete._estado = data.get("estado", "pendiente")
        return paquete


class PaqueteExpress(Paquete):
    def __init__(self, peso: float, precio_base: float, recargo: float = 0.5):
        super().__init__(peso, precio_base)
        self.recargo = recargo

    def calcular_precio(self):
        return self.precio_base * (1 + self.recargo)

    def __str__(self):
        return f"Paquete Express: {super().__str__()} - Recargo: {self.recargo * 100:.0f}%"

    def to_dict(self):
        data = super().to_dict()
        data["recargo"] = self.recargo
        return data


class PaqueteEstandar(Paquete):
    def __init__(self, peso: float, precio_base: float):
        super().__init__(peso, precio_base)

    def __str__(self):
        return f"Paquete Estándar: {super().__str__()}"


class Envio:
    def __init__(self, cliente: Cliente, paquete: Paquete):
        self.cliente = cliente  # Asociación: cliente existe fuera
        self.paquete = paquete  # Agregación: paquete asociado

    def calcular_precio(self):
        return self.paquete.calcular_precio()

    def __str__(self):
        return f"Envio a {self.cliente.nombre} - {self.paquete} - Precio total: ${self.calcular_precio():.2f}"

    def to_dict(self):
        return {
            "cliente": self.cliente.to_dict(),
            "paquete": self.paquete.to_dict(),
        }

    @classmethod
    def from_dict(cls, data):
        cliente = Cliente.from_dict(data["cliente"])
        paquete = Paquete.from_dict(data["paquete"])
        return cls(cliente, paquete)


class SistemaEnvios:
    def __init__(self):
        self.clientes = []
        self.envios = []

    def agregar_cliente(self, cliente: Cliente):
        if any(c.nombre == cliente.nombre for c in self.clientes):
            raise ValueError("Cliente ya registrado.")
        self.clientes.append(cliente)

    def buscar_cliente(self, nombre: str):
        for cliente in self.clientes:
            if cliente.nombre.lower() == nombre.lower():
                return cliente
        return None

    def crear_envio(self, nombre_cliente: str, paquete: Paquete):
        cliente = self.buscar_cliente(nombre_cliente)
        if not cliente:
            raise ValueError("Cliente no encontrado.")
        envio = Envio(cliente, paquete)
        self.envios.append(envio)
        return envio

    def mostrar_envios(self):
        if not self.envios:
            print("No hay envíos registrados.")
            return
        for envio in self.envios:
            print(envio)

    def totales_ventas(self):
        total_estandar = 0
        total_express = 0
        for envio in self.envios:
            if isinstance(envio.paquete, PaqueteExpress):
                total_express += envio.calcular_precio()
            elif isinstance(envio.paquete, PaqueteEstandar):
                total_estandar += envio.calcular_precio()
        print(f"Total ventas Paquetes Estándar: ${total_estandar:.2f}")
        print(f"Total ventas Paquetes Express: ${total_express:.2f}")

    def guardar_archivo(self, filename="envios.json"):
        datos = {
            "clientes": [cliente.to_dict() for cliente in self.clientes],
            "envios": [envio.to_dict() for envio in self.envios],
        }
        with open(filename, "w") as f:
            json.dump(datos, f, indent=4)
        print(f"Datos guardados en {filename}")

    def cargar_archivo(self, filename="envios.json"):
        try:
            with open(filename, "r") as f:
                datos = json.load(f)
            self.clientes = [Cliente.from_dict(c) for c in datos.get("clientes", [])]
            self.envios = [Envio.from_dict(e) for e in datos.get("envios", [])]
            print(f"Datos cargados desde {filename}")
        except FileNotFoundError:
            print(f"No se encontró el archivo {filename}, iniciando con datos vacíos.")


def menu():
    sistema = SistemaEnvios()
    sistema.cargar_archivo()

    while True:
        print("\n--- Menú Sistema de Gestión de Paquetería ---")
        print("1. Registrar cliente")
        print("2. Crear envío")
        print("3. Cambiar estado de paquete")
        print("4. Mostrar envíos")
        print("5. Mostrar totales de ventas")
        print("6. Guardar datos")
        print("7. Cargar datos")
        print("0. Salir")

        opcion = input("Seleccione una opción: ").strip()

        if opcion == "1":
            try:
                nombre = input("Nombre cliente: ").strip()
                calle = input("Calle: ").strip()
                ciudad = input("Ciudad: ").strip()
                codigo_postal = input("Código postal: ").strip()
                direccion = Direccion(calle, ciudad, codigo_postal)
                cliente = Cliente(nombre, direccion)
                sistema.agregar_cliente(cliente)
                print("Cliente registrado con éxito.")
            except ValueError as e:
                print(f"Error: {e}")

        elif opcion == "2":
            try:
                nombre_cliente = input("Nombre del cliente para envío: ").strip()
                tipo_paquete = input("Tipo de paquete (express/estandar): ").strip().lower()
                peso = float(input("Peso del paquete (kg): "))
                precio_base = float(input("Precio base del paquete: "))

                if tipo_paquete == "express":
                    recargo = 0.5  # fijo, pero se podría pedir al usuario
                    paquete = PaqueteExpress(peso, precio_base, recargo)
                elif tipo_paquete == "estandar":
                    paquete = PaqueteEstandar(peso, precio_base)
                else:
                    print("Tipo de paquete inválido.")
                    continue

                envio = sistema.crear_envio(nombre_cliente, paquete)
                print(f"Envío creado:\n{envio}")

            except ValueError as e:
                print(f"Error: {e}")
            except Exception:
                print("Datos inválidos.")

        elif opcion == "3":
            try:
                if not sistema.envios:
                    print("No hay envíos para actualizar.")
                    continue

                print("Envíos disponibles:")
                for i, envio in enumerate(sistema.envios):
                    print(f"{i + 1}. {envio}")

                idx = int(input("Seleccione el número del envío para cambiar estado: ")) - 1
                if idx < 0 or idx >= len(sistema.envios):
                    print("Número inválido.")
                    continue

                nuevo_estado = input("Nuevo estado (pendiente/en tránsito/entregado): ").strip().lower()
                sistema.envios[idx].paquete.set_estado(nuevo_estado)
                print("Estado actualizado.")

            except ValueError as e:
                print(f"Error: {e}")

        elif opcion == "4":
            sistema.mostrar_envios()

        elif opcion == "5":
            sistema.totales_ventas()

        elif opcion == "6":
            sistema.guardar_archivo()

        elif opcion == "7":
            sistema.cargar_archivo()

        elif opcion == "0":
            print("Saliendo del sistema.")
            break

        else:
            print("Opción inválida. Intente de nuevo.")


if __name__ == "__main__":
    menu()





No se encontró el archivo envios.json, iniciando con datos vacíos.

--- Menú Sistema de Gestión de Paquetería ---
1. Registrar cliente
2. Crear envío
3. Cambiar estado de paquete
4. Mostrar envíos
5. Mostrar totales de ventas
6. Guardar datos
7. Cargar datos
0. Salir
Saliendo del sistema.
